In [166]:
pip install galois

In [167]:
# Needed libraries
import secrets
import math
import numpy as np
import hashlib
import galois

# Generate private seed

In [168]:
private_seed = secrets.token_bytes(32)
print(private_seed)

b'\x89e\x91\xeaW\x9f\x94\x0b\x03\xcb/\xc9\xa987.#\x80\x19\xc3=#\xa1\x06\xb8\xd2\xb8\xbd\xd6\xc0G\x8c'


# SHAKE A.K.A function H (Squeeze public_seed and T)

In [169]:
v = 197
m = 57
r = 7

## SHAKE using hashlib

In [170]:
if(m<=57):
  h_shake = hashlib.shake_128(private_seed)
else:
  h_shake = hashlib.shake_256(private_seed)
private_sponge = h_shake.digest(32+math.ceil(m/8)*v)
print(private_sponge)

b'\xf9\xf3\xc4Z73\'\x93\x1f\xf2Z Oe\xb1w\x18\xca\x87\x98\x16\xcc\xc0\x1cs\xd3\xa6\xe5^{\x0c\xcd\xb2\x92\x9a1\x02k\x8eo\xbal\xacWs\x08Hy\xdb\x8e\x0b\x9fl\xb5;\x97\xd7\xb3e\xeb?\t\x84J\xbf\x04;\x12\x82c5\xf9\x86\xd5\x9c\x7f/\xbe\n)*\x83\xc5I\x93\x8f@6\xb6\t\xdd\x84\xd94D[0\\\x002\xcd\x8b\x00\x8a\xe4]1\xee\xcb\xaau\x88e\x86\xaeX\xd3Y\xf3\x82\xdaB\x91\xca}}\xa2\x82`\xb2\x89\xf5\t\x10@\xbd\xcb\xb4\r*\xee\xbf3\x8f\xabPe\xa8\x87Q\x10\x97in\x8b\xc2V$\xddkZ\xfe\x9b#Z\x1f\x15*\x07\xb2\xdb\xaeI\x0b\x95\xd6 }\xeb\x97\xf0\xdcBj\xd6c\x04E\x05\xbe\xb6\xa8\xa9l\x8d\xd7\xa4J\xed\x9f/H\xa8\x88}\xb9\xc7(\x97\t\x90\xd9)*\xc2H\xe1QGx\x8c\xec\xad#\xe9c%X\xabe\xa5J\xea\x8b\xa5\x81\xb2\x99\xb1=\xaf\xbb.\xe3\xf19\x81Z\x93+y\x17\x16g\xaf\xd4\xaft\x97h\xeb\x16\xf8P\xd8\xee3\x9e\xd0\xcb\x94\xb78\xfd\xa0\x13{i]\x93c\x9f\xb9\xb83_\x92nW\xb0\xb4h~\xf17Q\x0bzt\xfa\xab\x86\xb6)\xea\xf2+l\'/\xcbE\xaay,\xec\xff*\x8e\xb5\xf1\x1d\x8c\xab!a\x86\xb3\x12\x83G\x0fl)\x01J\x82\x13\xd6?\xfc`\xe0s\xbd\x8e\xc0\x9bA\xc2hk$\xcfg\x13

#Extract public seed

In [171]:
public_seed = private_sponge[:32]
print(public_seed)

b"\xf9\xf3\xc4Z73'\x93\x1f\xf2Z Oe\xb1w\x18\xca\x87\x98\x16\xcc\xc0\x1cs\xd3\xa6\xe5^{\x0c\xcd"


#Extract T

In [172]:
def int8_to_bits(intByte:int)->list:
  """Get the first 8 bits of an integer as an array.

  The function takes one integer byte, extracts it's bits
  and returns an array containing the bits starting from
  the most significant bit.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  list
      An array with the first 8 bits of the integer
      starting from the most significant one.
  """
  storage = []
  for i in range(8):
    storage.insert(0,(intByte>>i)&1)
  return storage

In [173]:
def tMatrixRowGen(byteString:bytes,neededBits:int)->list:
  """Generate one row of the T matrix.

  The function takes one byte-string and extracts the specified
  ammount of bits (neededBits) to fill one row. When the ammount
  of bits doesn't fit completely in a byte, the most significant
  bits of the last byte are ignored.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the row.
  neededBits : int
    The ammount of bits a row needs to have.

  Returns
  -------
  list
      An array representing one row of the T matrix
  """
  row = []
  while(neededBits>8):
    row+=int8_to_bits(byteString[0])
    byteString = byteString[1:]
    neededBits = neededBits-8
  if(neededBits>0):
    row+=int8_to_bits(byteString[0])[-neededBits:]
  return row

def tMatrixGenerator(byteString:bytes,v:int,m:int)->np.ndarray:
  """Generate the T matrix.

  The function takes one byte-string and generates v rows
  each one with m bits extracted sequentially from a byteString
  using the tMatrixRowGen() function.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.
  v : int
    The ammount rows of the matrix.
  m : int
    The ammount columns of the matrix.

  Returns
  -------
  list
      An array representing the T matrix
  """
  mdivided8 =math.ceil(m/8)
  matrix = []
  for i in range(v):
    matrix.append(tMatrixRowGen(byteString,m))
    byteString = byteString[mdivided8:]
  return np.array(matrix)

In [174]:
T_base = private_sponge[32:]
T = tMatrixGenerator(T_base,v,m)
print(T)

[[1 0 1 ... 1 0 1]
 [1 0 1 ... 0 0 1]
 [1 1 0 ... 1 1 1]
 ...
 [1 0 0 ... 0 1 0]
 [1 0 1 ... 1 1 1]
 [1 0 1 ... 0 0 1]]


# SHAKE (squeeze public map)

In [175]:
def subMatrixGenerator(byteString:bytes)->np.ndarray:
  """Generate a part of the L or Q1 matrix.

  The function takes one byte-string and returns one 16 row part
  of the L or Q1 matrix. It must be noted that the rows are generated as columns
  at first and then are transposed.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.

  Returns
  -------
  np.ndarray
      A matrix
  """
  matrix = []
  while(len(byteString)>0):
    matrix.append(int8_to_bits(byteString[0])+int8_to_bits(byteString[1]))
    byteString = byteString[2:]
  return np.array(matrix).T

In [176]:
n = m+v
g_equation = 2+2*n+v*(v+1)+2*v*m
mdivided16 = math.ceil(m/16)

#first g function iteration
g_shake = hashlib.shake_128(public_seed+bytes.fromhex('00'))
public_sponge = g_shake.digest(g_equation)
C_base=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
public_sponge = public_sponge[2:]
# L and Q1 have more than one dimension, so they are initilized
# differently
L_base = subMatrixGenerator(public_sponge[:2*n])
public_sponge = public_sponge[2*n:]
Q1_base = subMatrixGenerator(public_sponge)

#following g function iterations
for i in range(mdivided16):
  g_shake = hashlib.shake_128(public_seed+bytes.fromhex(f'0{i}'))
  public_sponge = g_shake.digest(g_equation)
  C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
  public_sponge = public_sponge[2:]
  L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
  public_sponge = public_sponge[2*n:]
  Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))

### Obtaining C

In [177]:
# we take the m bits (starting from the last generated bit) we need from
# the total of bits generated
C = C_base[-m:]
C = np.array(C)
print(C)

[1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 0]


### Obtaining L

In [178]:
#We discard the upper rows to get the specified dimensions
L = L_base[-m:]
print(L)

[[0 0 1 ... 1 1 0]
 [1 1 1 ... 1 0 0]
 [0 0 0 ... 0 1 1]
 ...
 [0 1 0 ... 1 1 1]
 [0 1 0 ... 0 0 0]
 [1 0 1 ... 1 0 0]]


### Obtaining $$Q_{1}$$

In [179]:
#we discard the upper row to get the specified dimensions
Q1=Q1_base[-m:]
print(Q1)

[[0 0 0 ... 1 1 0]
 [1 0 1 ... 0 1 0]
 [0 1 0 ... 1 0 0]
 ...
 [0 1 0 ... 1 1 0]
 [1 0 0 ... 0 1 1]
 [0 0 0 ... 1 0 1]]


# find $$Q_2$$

In [180]:
def findPk1(k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk1 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-v matrix representing the part of pk that is
      quadratic in the vinegar variables.
  """
  Pk1 = np.zeros([v,v],dtype=np.int8)
  column = 0
  for i in range(v):
    for j in range(i,v):
      Pk1[i,j]= q1[k,column]
      column+=1
    column+=m
  return Pk1

In [181]:
def findPk2(k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk2 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-m matrix representing the part of pk that is bilinear in the
      vinegar variables and the oil variables.
  """
  Pk2 = np.zeros([v,m],dtype=np.int8)
  column = 0
  for i in range(v):
    column += v-i
    for j in range(m):
      Pk2[i,j]= q1[k,column]
      column+=1
  return Pk2

In [182]:
def findQ2(Q1:np.ndarray,T:np.ndarray)->np.ndarray:
  """An implementation of the findQ2 algorithm of the LUOV specification.

  Parameters
  ----------
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.
  t : np.ndarray
    A v-by-m matrix.

  Returns
  -------
  np.ndarray
      The second part of Macaulay matrix for quadratic part of P.
  """
  # we use galois to prevent the results of the operations from
  # being out oh the finite field
  GF = galois.GF(2)
  T = GF(T)

  Q2 = GF(np.zeros([m,int(m*(m+1)/2)],dtype=np.int8))
  for k in range(m):
    Pk1 = GF(findPk1(k,Q1))
    Pk2 = GF(findPk2(k,Q1))
    Pk3 = -np.transpose(T)@Pk1@T+np.transpose(T)@Pk2
    column = 0
    for i in range(m):
      Q2[k,column]= Pk3[i,i]
      column += 1
      for j in range(i+1,m):
        Q2[k,column] = Pk3[i,j] + Pk3[j,i]
        column+=1
  return Q2

In [183]:
Q2 = findQ2(Q1,T)
print(Q2)
print(np.shape(Q2))

[[0 1 1 ... 0 0 1]
 [0 1 0 ... 1 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 1 ... 0 1 1]
 [0 0 1 ... 1 1 1]
 [0 1 0 ... 1 1 1]]
(57, 1653)


In [184]:
public_key = public_seed
for j in range(np.shape(Q2)[1]):
  public_key += bytes(Q2[:,j])
print(public_key)
print(len(public_key))

b"\xf9\xf3\xc4Z73'\x93\x1f\xf2Z Oe\xb1w\x18\xca\x87\x98\x16\xcc\xc0\x1cs\xd3\xa6\xe5^{\x0c\xcd\x00\x00\x00\x01\x01\x01\x00\x00\x01\x01\x00\x01\x01\x01\x00\x01\x01\x00\x01\x01\x01\x00\x01\x00\x01\x00\x00\x01\x01\x01\x00\x01\x00\x01\x00\x00\x01\x00\x00\x00\x01\x00\x01\x01\x00\x00\x01\x00\x00\x00\x00\x01\x01\x00\x01\x00\x00\x01\x01\x00\x00\x01\x01\x00\x00\x00\x00\x01\x01\x00\x01\x00\x01\x00\x01\x01\x01\x01\x00\x00\x00\x01\x01\x01\x00\x00\x01\x00\x01\x01\x00\x00\x01\x00\x01\x00\x01\x00\x01\x00\x00\x00\x01\x01\x00\x00\x01\x01\x00\x01\x00\x00\x00\x01\x01\x00\x00\x01\x00\x00\x01\x00\x00\x01\x00\x00\x01\x00\x01\x01\x01\x00\x00\x00\x00\x01\x01\x00\x00\x00\x00\x00\x00\x01\x00\x00\x01\x00\x01\x01\x01\x00\x01\x00\x00\x00\x01\x00\x01\x00\x00\x00\x01\x01\x00\x01\x00\x01\x01\x01\x00\x01\x01\x01\x00\x00\x00\x01\x01\x00\x01\x01\x01\x00\x00\x00\x01\x00\x00\x00\x01\x01\x01\x01\x01\x01\x00\x01\x00\x00\x01\x00\x00\x01\x00\x01\x00\x01\x01\x00\x00\x00\x00\x01\x01\x00\x00\x00\x00\x00\x00\x01\x01\x01\x01\x00\x